<a href="https://colab.research.google.com/github/danadler-dev/voice_leading/blob/main/VoiceLeading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from enum import Enum

_chord_tone = {1:0, 3:1, 5:2}

class Scale(Enum):
    MAJOR = 1
    H_MINOR = 2
    M_MINOR = 3

############################################################
# Class to generate 3,4,5 part diatonic chords for any scale
# For example: 4 part chords in F Major:
# c = Diatonic("F", Scale.MAJOR)
# c.chords(4)
############################################################

class Diatonic:
  _tonic = '?'
  _scale = []
  _notes_flat  = ["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B"]
  _Notes_flat  = pd.Series(_notes_flat)

  _notes_sharp = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
  _Notes_sharp = pd.Series(_notes_sharp)

  def _mk_scale(self, intervals):
    sharp = lambda t: ('#' in t) or (t in ["D", "E", "G", "A", "B"])
    notes = self._notes_sharp if sharp(self._tonic) else self._notes_flat
    Notes = self._Notes_sharp if sharp(self._tonic) else self._Notes_flat
    offset = notes.index(self.tonic)
    ivals = [(x + offset) % 12 for x in intervals]
    return list(Notes[ivals])

  def _major(self):
    return self._mk_scale([0,2,4,5,7,9,11])

  def _mel_minor(self):
    return self._mk_scale([0,2,3,5,7,9,11])

  def _har_minor(self):
    return self._mk_scale([0,2,3,5,7,8,10])

  def __init__(self, tonic:str, kind:Scale ):
    self.tonic = tonic
    if kind == Scale.MAJOR:
      self._scale = self._major()
    elif kind == Scale.M_MINOR:
      self._scale = self._mel_minor()
    else:
      self._scale = self._har_minor() 

  def chords(self, num=3):
    next = lambda x, n=1: self._scale[(self._scale.index(x)+n) % len(self._scale)]
    return [tuple([next(c,2*i) for i in range(num)]) for c in self._scale]


In [2]:
# 4-part diatonic chords in F melodic minor
c = Diatonic("F", Scale.M_MINOR)
assert c.chords(4) == [
 ('F', 'Ab', 'C', 'E'),
 ('G', 'Bb', 'D', 'F'),
 ('Ab', 'C', 'E', 'G'),
 ('Bb', 'D', 'F', 'Ab'),
 ('C', 'E', 'G', 'Bb'),
 ('D', 'F', 'Ab', 'C'),
 ('E', 'G', 'Bb', 'D')]

In [3]:
# 5-part diatonic chords in C major
c = Diatonic("C", Scale.MAJOR)
assert c.chords(5) == [
 ('C', 'E', 'G', 'B', 'D'),
 ('D', 'F', 'A', 'C', 'E'),
 ('E', 'G', 'B', 'D', 'F'),
 ('F', 'A', 'C', 'E', 'G'),
 ('G', 'B', 'D', 'F', 'A'),
 ('A', 'C', 'E', 'G', 'B'),
 ('B', 'D', 'F', 'A', 'C') 
]

In [4]:
c = Diatonic("C", Scale.MAJOR)
chords = c.chords(3)

In [5]:
pc = (5,3,1) # cycle: 5->3->1->5...
def perm(*args):
  return tuple([pc[(pc.index(a)+1) % len(pc)] for a in args[0]])


In [6]:
start = (1,3,5)
assert perm(perm(perm(start))) == start

In [7]:
# invert a chord like (C,E, G) by a permutation like (3,1,5)
def invert(chord, prm):
  loc = tuple([_chord_tone[t] for t in prm]) 
  return tuple([chord[loc[i]] for i in range(len(prm))])


In [8]:
assert invert(('C','E','G'), (3,5,1)) == ('E', 'G', 'C')

In [9]:
# The 6 cycles as defined in the intro
Cycle2 = []; Cycle3 = []; Cycle4 = []; Cycle5 = []; Cycle6 = []; Cycle7 = []
Cycles = [Cycle2, Cycle3, Cycle4, Cycle5, Cycle6, Cycle7 ]
for i in range(6): # 6 cycles
  j=0
  for k in range(len(chords)):
    Cycles[i].append(chords[j])
    j = (j + i + 1) % len(chords)

In [10]:
pc = (5,3,1) # cycle: 5->3->1->5...
start = [(1,3,5), (1,5,3)] # close voicing, spread voicing
for p in start:
  for i in range(3):
    for c in Cycle2:
      print(invert(c, p))
      p = perm(p)
    print('---------------')
  print('================')

('C', 'E', 'G')
('A', 'D', 'F')
('G', 'B', 'E')
('F', 'A', 'C')
('D', 'G', 'B')
('C', 'E', 'A')
('B', 'D', 'F')
---------------
('G', 'C', 'E')
('F', 'A', 'D')
('E', 'G', 'B')
('C', 'F', 'A')
('B', 'D', 'G')
('A', 'C', 'E')
('F', 'B', 'D')
---------------
('E', 'G', 'C')
('D', 'F', 'A')
('B', 'E', 'G')
('A', 'C', 'F')
('G', 'B', 'D')
('E', 'A', 'C')
('D', 'F', 'B')
---------------
('C', 'G', 'E')
('A', 'F', 'D')
('G', 'E', 'B')
('F', 'C', 'A')
('D', 'B', 'G')
('C', 'A', 'E')
('B', 'F', 'D')
---------------
('G', 'E', 'C')
('F', 'D', 'A')
('E', 'B', 'G')
('C', 'A', 'F')
('B', 'G', 'D')
('A', 'E', 'C')
('F', 'D', 'B')
---------------
('E', 'C', 'G')
('D', 'A', 'F')
('B', 'G', 'E')
('A', 'F', 'C')
('G', 'D', 'B')
('E', 'C', 'A')
('D', 'B', 'F')
---------------


In [11]:
pc = (5,3,1) # cycle: 5->3->1->5...
start = [(1,3,5), (1,5,3)] # close voicing, spread voicing
for p in start:
  for i in range(3):
    for c in Cycle4:
      print(invert(c, p))
      p = perm(p)
    print('---------------')
  print('================')

('C', 'E', 'G')
('C', 'F', 'A')
('D', 'F', 'B')
('E', 'G', 'B')
('E', 'A', 'C')
('F', 'A', 'D')
('G', 'B', 'D')
---------------
('G', 'C', 'E')
('A', 'C', 'F')
('B', 'D', 'F')
('B', 'E', 'G')
('C', 'E', 'A')
('D', 'F', 'A')
('D', 'G', 'B')
---------------
('E', 'G', 'C')
('F', 'A', 'C')
('F', 'B', 'D')
('G', 'B', 'E')
('A', 'C', 'E')
('A', 'D', 'F')
('B', 'D', 'G')
---------------
('C', 'G', 'E')
('C', 'A', 'F')
('D', 'B', 'F')
('E', 'B', 'G')
('E', 'C', 'A')
('F', 'D', 'A')
('G', 'D', 'B')
---------------
('G', 'E', 'C')
('A', 'F', 'C')
('B', 'F', 'D')
('B', 'G', 'E')
('C', 'A', 'E')
('D', 'A', 'F')
('D', 'B', 'G')
---------------
('E', 'C', 'G')
('F', 'C', 'A')
('F', 'D', 'B')
('G', 'E', 'B')
('A', 'E', 'C')
('A', 'F', 'D')
('B', 'G', 'D')
---------------


In [12]:
pc = (5,1,3) # cycle: 5->3->1->5...
start = [(1,3,5), (1,5,3)] # close voicing, spread voicing
for p in start:
  for i in range(3):
    for c in Cycle6:
      print(invert(c, p))
      p = perm(p)
    print('---------------')
  print('================')

('C', 'E', 'G')
('C', 'E', 'A')
('C', 'F', 'A')
('D', 'F', 'A')
('D', 'F', 'B')
('D', 'G', 'B')
('E', 'G', 'B')
---------------
('E', 'G', 'C')
('E', 'A', 'C')
('F', 'A', 'C')
('F', 'A', 'D')
('F', 'B', 'D')
('G', 'B', 'D')
('G', 'B', 'E')
---------------
('G', 'C', 'E')
('A', 'C', 'E')
('A', 'C', 'F')
('A', 'D', 'F')
('B', 'D', 'F')
('B', 'D', 'G')
('B', 'E', 'G')
---------------
('C', 'G', 'E')
('C', 'A', 'E')
('C', 'A', 'F')
('D', 'A', 'F')
('D', 'B', 'F')
('D', 'B', 'G')
('E', 'B', 'G')
---------------
('E', 'C', 'G')
('E', 'C', 'A')
('F', 'C', 'A')
('F', 'D', 'A')
('F', 'D', 'B')
('G', 'D', 'B')
('G', 'E', 'B')
---------------
('G', 'E', 'C')
('A', 'E', 'C')
('A', 'F', 'C')
('A', 'F', 'D')
('B', 'F', 'D')
('B', 'G', 'D')
('B', 'G', 'E')
---------------


In [13]:
Cycle2 = []; Cycle3 = []; Cycle4 = []; Cycle5 = []; Cycle6 = []; Cycle7 = []
Cycles = [Cycle2, Cycle3, Cycle4, Cycle5, Cycle6, Cycle7 ]
for i in range(6): # 6 cycles
  j=0
  for k in range(len(chords)):
    Cycles[i].append(chords[j])
    j = (j + i + 1) % len(chords)


In [14]:
Cycle5

[('C', 'E', 'G'),
 ('G', 'B', 'D'),
 ('D', 'F', 'A'),
 ('A', 'C', 'E'),
 ('E', 'G', 'B'),
 ('B', 'D', 'F'),
 ('F', 'A', 'C')]